# Chapter 10-01: GPU 메모리 프로파일링 (Memory Profiling)

## 학습 목표
- GPU VRAM을 차지하는 **4가지 구성요소**를 수식 레벨에서 정확히 파악하고, 모델 아키텍처만 보고 필요 메모리를 스스로 계산할 수 있다
- **Mixed Precision 학습** (FP16/BF16 + FP32 마스터 가중치)의 필요성과 메모리-속도 Trade-off를 분석한다
- FP8 / BF16 / FP16 / FP32 데이터타입의 **수치 범위와 정밀도 차이**를 코드로 직접 확인한다
- 실제 Transformer 모델의 **Activation Memory**를 추정하는 공식을 도출하고, Sequence Length에 따른 변화를 시각화한다

## 목차
1. [수학적 기초: 메모리 분류와 공식](#1.-수학적-기초)
2. [데이터타입별 범위와 정밀도 실험](#2.-데이터타입-실험)
3. [모델 설정별 메모리 계산기](#3.-메모리-계산기)
4. [Activation Memory 추정 및 시각화](#4.-Activation-Memory)
5. [메모리 최적화 옵션 비교](#5.-최적화-옵션-비교)
6. [정리 및 연습 문제](#6.-정리)

## 1. 수학적 기초

### GPU VRAM 4가지 구성요소

$$M_{VRAM} = M_{params} + M_{grad} + M_{optim} + M_{act}$$

| 구성요소 | 데이터타입 | 크기 | 설명 |
|---------|-----------|------|------|
| 모델 파라미터 $M_{params}$ | FP16 (Mixed) | $2P$ | 추론 및 Forward 에 사용 |
| 그래디언트 $M_{grad}$ | FP16 (Mixed) | $2P$ | Backward 시 계산 |
| 마스터 가중치 (Mixed만) | FP32 | $4P$ | 옵티마이저 업데이트 정밀도 보장 |
| Adam 1차 모멘트 $m$ | FP32 | $4P$ | 지수이동평균 그래디언트 |
| Adam 2차 모멘트 $v$ | FP32 | $4P$ | 지수이동평균 그래디언트² |
| **합계 (Mixed+Adam)** | | $\mathbf{16P}$ | |
| **Activation** $M_{act}$ | FP16 | $O(B \cdot S \cdot H \cdot L)$ | Batch, Seq, Hidden, Layer 크기 비례 |

### Mixed Precision의 핵심 회로

```
FP32 마스터 가중치
       │  (업데이트 δW 적용)       ← 정밀도 보장
       ▼
    FP32 마스터 가중치
       │  (FP32→FP16 캐스팅)
       ▼
  FP16 가중치 ──→ Forward (FP16: 2배 빠름) ──→ FP16 활성화
                                                      │
                                                      ▼
                 Backward (FP16) ◄──────────── FP16 그래디언트
                      │  (Loss Scale: NaN 방지)
                      ▼
               FP32로 누적 → Adam 업데이트 → FP32 마스터 가중치
```

### Transformer Activation Memory (레이어당)

$$M_{act, layer} \approx B \cdot S \cdot H \cdot \underbrace{(4 + 2h/H + 2S/H)}_{\text{attention 항}} \cdot \text{dtype bytes}$$

단순 근사: $M_{act} \approx B \cdot S \cdot H \cdot 12 \cdot L \cdot \text{bytes}$  
(여기서 $B$=배치, $S$=시퀀스, $H$=히든, $L$=레이어)

---

### 🐣 초등학생을 위한 친절 설명!

#### 💾 GPU 메모리가 왜 꽉 차나요?

> AI 훈련 시 GPU에 올라가는 것들:
> - 📚 **교과서(모델 가중치)**: 가장 기본 필수품
> - 📝 **오답 노트(그래디언트)**: 어디서 틀렸는지 기록
> - 📊 **성적 기록부(Adam 상태)**: 과거 학습 흐름 기억
> - 🖍️ **연습장(Activation)**: 중간 계산 결과 (배치가 클수록 많아짐!)
>
> 이 4가지 중 **Adam 상태(옵티마이저)가 50%** 이상 차지해요!

#### 🎭 FP16과 FP32의 차이는?

> - **FP32(4바이트)**: 아주 정밀한 계산 (소수점 아래 7자리)
> - **FP16(2바이트)**: 덜 정밀하지만 절반 메모리 (소수점 아래 3자리)
> - **꼼수(Mixed Precision)**: 빠른 계산은 FP16으로, 중요한 업데이트는 FP32로!

### 📝 연습 문제

#### 문제 1: 학습 메모리 계산

Llama-2 13B (13,000,000,000 파라미터)를 Mixed Precision + Adam으로 학습할 때:
- 모델 가중치만의 FP16 메모리는?
- 전체 학습 메모리는?
- A100 80GB GPU 몇 대가 필요한가? (70% 활용 가정)

<details>
<summary>💡 풀이 확인</summary>

$P = 13 \times 10^9$

- FP16 가중치: $2 \times 13 \times 10^9 = 26$ GB
- 전체 학습: $16 \times 13 = 208$ GB
- A100 80GB 70% = 56GB → $\lceil 208/56 \rceil = \mathbf{4}$대

(Activation 제외. 실제로는 더 필요)
</details>

#### 문제 2: FP16 오버플로우 위험

FP16의 최대값은 65504입니다.  
`attention_scores = Q @ K.T / sqrt(d_k)`에서 $d_k=64$일 때,  
Q, K 각 원소가 10.0이면 attention score의 최대값은 얼마인가?  
이것이 FP16에서 안전한가?

<details>
<summary>💡 풀이 확인</summary>

$Q = K = 10.0$, $d_k = 64$, 벡터 길이 $d_k$  
$\text{score} = \frac{Q \cdot K}{\sqrt{64}} = \frac{10 \times 10 \times 64}{8} = \frac{6400}{8} = 800$

800 < 65504 → 이 경우는 안전하지만, $d_k$가 크거나 값이 크면 오버플로우 위험!  
→ 이것이 **BF16(FP32와 동일한 지수 범위)**가 선호되는 이유
</details>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import struct

print("TensorFlow 버전:", tf.__version__)
tf.random.set_seed(42)
np.random.seed(42)

## 2. 데이터타입 실험

In [ ]:
# ---------------------------------------------------
# 데이터타입별 수치 범위, 정밀도, 메모리 비교
# ---------------------------------------------------

import tensorflow as tf
import numpy as np

dtype_info = {
    'FP32 (float32)': {
        'tf_dtype': tf.float32,
        'bytes': 4, 'exp_bits': 8, 'mantissa_bits': 23,
        'max': 3.4e38, 'min_pos': 1.18e-38
    },
    'BF16 (bfloat16)': {
        'tf_dtype': tf.bfloat16,
        'bytes': 2, 'exp_bits': 8, 'mantissa_bits': 7,
        'max': 3.4e38, 'min_pos': 1.18e-38
    },
    'FP16 (float16)': {
        'tf_dtype': tf.float16,
        'bytes': 2, 'exp_bits': 5, 'mantissa_bits': 10,
        'max': 65504, 'min_pos': 6.1e-5
    },
}

print(f"{'타입':<20} {'바이트':>5} {'지수비트':>8} {'가수비트':>8} {'최댓값':>12} {'최솟값(+)':>12} {'FP32 대비 메모리'}")  
print("-" * 85)
for name, info in dtype_info.items():
    savings = (1 - info['bytes'] / 4) * 100
    print(f"{name:<20} {info['bytes']:>5} {info['exp_bits']:>8} {info['mantissa_bits']:>8}"
          f" {info['max']:>12.2e} {info['min_pos']:>12.2e} "
          f"{'동일' if savings==0 else f'-{savings:.0f}%'}")

print("\n=== 정밀도 손실 실험 ===")
test_values = [1.0, 1.0 + 1e-3, 1.0 + 1e-4, 3.14159265358979, 65504.0, 65505.0]

print(f"{'원본 FP32값':>22} | {'BF16로 변환':>15} | {'FP16로 변환':>15} | {'설명'}")
print("-" * 80)
for v in test_values:
    orig    = tf.constant(v, dtype=tf.float32)
    in_bf16 = tf.cast(tf.cast(orig, tf.bfloat16), tf.float32).numpy()
    in_fp16 = tf.cast(tf.cast(orig, tf.float16), tf.float32).numpy()
    note = ''
    if v == 65505.0: note = '← FP16 최댓값 초과!'
    elif abs(v - in_fp16) > 1e-2: note = '← FP16 정밀도 손실'
    print(f"{v:>22.10f} | {in_bf16:>15.10f} | {in_fp16:>15.6f} | {note}")

print("\n→ BF16은 FP32와 동일한 지수 범위 → 오버플로우 없음")
print("→ FP16은 최댓값이 65504로 제한 → 오버플로우 위험 (Loss Scaling 필요)")

In [ ]:
# ---------------------------------------------------
# FP16 오버플로우 시뮬레이션 (Loss Scaling의 필요성)
# ---------------------------------------------------

# 극단적으로 작은 그래디언트 → FP16에서 언더플로우
tiny_grad = tf.constant(1e-8, dtype=tf.float32)
tiny_in_fp16 = tf.cast(tiny_grad, tf.float16).numpy()
print(f"작은 그래디언트 {1e-8} → FP16: {tiny_in_fp16}  ← 언더플로우! (0이 됨)")

# Loss Scaling: 그래디언트를 scale 배 키워서 저장, 나중에 나눔
loss_scale = 2048.0
scaled_grad = tiny_grad * loss_scale
scaled_in_fp16 = tf.cast(scaled_grad, tf.float16).numpy()
restored = scaled_in_fp16 / loss_scale

print(f"\n[Loss Scaling 적용] scale = {loss_scale}")
print(f"  원본: {tiny_grad.numpy():.2e}")
print(f"  스케일 후 FP16 저장: {scaled_in_fp16}")
print(f"  스케일 제거 복원:    {restored:.2e}")
print(f"  복원 성공: {'✅' if abs(restored - tiny_grad.numpy()) < 1e-10 else '❌'}")

print("\n→ Loss Scaling이 FP16 학습의 핵심 기법!")
print("   TF에서는 tf.keras.mixed_precision.LossScaleOptimizer 사용")

## 3. 메모리 계산기

In [ ]:
# ---------------------------------------------------
# 종합 학습 메모리 계산기
# ---------------------------------------------------

def compute_memory_breakdown(params_B, batch_size, seq_len, hidden_dim,
                             n_layers, n_heads, optimizer='adam',
                             dtype='fp16', act_checkpoint=False):
    """
    Transformer 모델의 전체 학습 메모리를 항목별로 분해.
    
    Returns: 항목별 메모리 딕셔너리 (GB 단위)
    """
    P = params_B * 1e9
    dty = 2 if dtype in ('fp16', 'bf16') else 4

    # ── 정적 메모리 (Param, Grad, Optimizer) ────────────
    param_mem  = P * dty / 1e9
    grad_mem   = P * dty / 1e9
    master_mem = (P * 4 / 1e9) if dtype in ('fp16', 'bf16') else 0.0

    optim_cfg = {'sgd': 0, 'momentum': P*4/1e9, 'adam': P*8/1e9}
    optim_mem = optim_cfg.get(optimizer, P*8/1e9)

    # ── Activation Memory (Transformer 근사) ─────────────
    # 핵심 항: MHA Q,K,V 프로젝션 + Attention Score + FFN
    # 레이어당: B × S × (12H) × bytes (근사)
    act_per_layer = batch_size * seq_len * hidden_dim * 12 * dty / 1e9
    if act_checkpoint:
        # Gradient Checkpointing: √L 레이어만 보존 (나머지 재계산)
        act_mem = act_per_layer * int(np.sqrt(n_layers))
    else:
        act_mem = act_per_layer * n_layers

    total = param_mem + grad_mem + master_mem + optim_mem + act_mem

    return {
        'param':   param_mem,
        'grad':    grad_mem,
        'master':  master_mem,
        'optim':   optim_mem,
        'act':     act_mem,
        'total':   total
    }


# ── 주요 LLM 메모리 계산 ────────────────────────────────
llms = [
    # (이름,   params_B, batch, seq,  H,     L,  heads)
    ('Llama-2 7B',   7,   4, 4096,  4096, 32,  32),
    ('Llama-2 13B', 13,   4, 4096,  5120, 40,  40),
    ('Llama-2 70B', 70,   2, 4096,  8192, 80,  64),
]

print(f"{'모델':<15} {'Param':>7} {'Grad':>6} {'Master':>7} {'Adam':>6} {'Act':>6} {'합계':>7} | H100 필요")
print("-" * 75)

for name, p, b, s, h, l, nh in llms:
    mb = compute_memory_breakdown(p, b, s, h, l, nh)
    n_gpu = max(1, int(np.ceil(mb['total'] / (80 * 0.7))))
    print(f"{name:<15} {mb['param']:>7.1f} {mb['grad']:>6.1f} {mb['master']:>7.1f}"
          f" {mb['optim']:>6.1f} {mb['act']:>6.1f} {mb['total']:>7.1f} GB | {n_gpu}대")

## 4. Activation Memory

In [ ]:
# ---------------------------------------------------
# Activation Memory vs Sequence Length 시각화
# ---------------------------------------------------

seq_lens = np.array([512, 1024, 2048, 4096, 8192, 16384, 32768])
params_B, hidden, layers, heads = 7, 4096, 32, 32
batch = 1

act_mems = []
for sl in seq_lens:
    mb = compute_memory_breakdown(params_B, batch, int(sl), hidden, layers, heads)
    act_mems.append(mb['act'])

static_mem = compute_memory_breakdown(params_B, batch, 512, hidden, layers, heads)['total'] \
           - compute_memory_breakdown(params_B, batch, 512, hidden, layers, heads)['act']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 왼쪽: Activation vs 총 메모리
ax1 = axes[0]
total_mems = [static_mem + a for a in act_mems]
ax1.fill_between(seq_lens, 0, [static_mem]*len(seq_lens), alpha=0.6, color='#1E88E5',
                 label=f'정적 메모리 (Param+Grad+Optim: {static_mem:.0f}GB)')
ax1.fill_between(seq_lens, [static_mem]*len(seq_lens), total_mems, alpha=0.6, color='#E53935',
                 label='Activation Memory (∝ Seq Length)')
ax1.axhline(y=80, color='k', ls='--', lw=2, label='H100 80GB')
ax1.axhline(y=80*0.7, color='gray', ls=':', lw=1.5, label='H100 56GB (70%)')

for sl, tm in zip(seq_lens, total_mems):
    ax1.annotate(f'{tm:.0f}GB', (sl, tm + 3), ha='center', fontsize=8, color='darkred')

ax1.set_xscale('log', base=2)
ax1.set_xticks(seq_lens)
ax1.set_xticklabels([f'{sl//1024}K' if sl>=1024 else str(sl) for sl in seq_lens])
ax1.set_xlabel('시퀀스 길이 (Sequence Length)', fontsize=11)
ax1.set_ylabel('메모리 (GB)', fontsize=11)
ax1.set_title(f'Llama-2 7B: 시퀀스 길이별 메모리\n(batch=1)', fontweight='bold')
ax1.legend(fontsize=8, loc='upper left')
ax1.grid(True, alpha=0.3)

# 오른쪽: Activation Memory 비율 추세
ax2 = axes[1]
act_ratios = [a / t * 100 for a, t in zip(act_mems, total_mems)]

ax2.bar(range(len(seq_lens)), act_ratios, color='#E53935', alpha=0.75, label='Activation 비율')
ax2.plot(range(len(seq_lens)), act_ratios, 'ko-', lw=2, ms=6)
ax2.set_xticks(range(len(seq_lens)))
ax2.set_xticklabels([f'{sl//1024}K' if sl>=1024 else str(sl) for sl in seq_lens])
ax2.set_xlabel('시퀀스 길이 (Sequence Length)', fontsize=11)
ax2.set_ylabel('Activation이 총 메모리에서 차지하는 비율 (%)', fontsize=11)
ax2.set_title('긴 시퀀스일수록\nActivation이 메모리 지배', fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

for i, (pct, sl) in enumerate(zip(act_ratios, seq_lens)):
    ax2.text(i, pct + 1.5, f'{pct:.0f}%', ha='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"결론:")
print(f"  • 짧은 시퀀스: 정적 메모리(파라미터 등) 지배")
print(f"  • 긴 시퀀스(4K+): Activation이 절반 이상 → Gradient Checkpointing 필수!")
print(f"  • Attention의 O(S²) 항 때문에 시퀀스 길이 4배 → Activation ≈ 4배 증가")

## 5. 최적화 옵션 비교

In [ ]:
# ---------------------------------------------------
# 메모리 최적화 기법별 효과 비교
# ---------------------------------------------------

def compute_scenario(params_B, b, s, h, l, nh, scenario):
    """다양한 메모리 최적화 시나리오 계산"""
    cfg = {
        '① FP32 + SGD':          {'dtype': 'fp32', 'optimizer': 'sgd', 'act_ckpt': False},
        '② FP32 + Adam':         {'dtype': 'fp32', 'optimizer': 'adam', 'act_ckpt': False},
        '③ FP16 + Adam':         {'dtype': 'fp16', 'optimizer': 'adam', 'act_ckpt': False},
        '④ FP16 + Adam + GC':    {'dtype': 'fp16', 'optimizer': 'adam', 'act_ckpt': True},
        '⑤ BF16 + Adam':         {'dtype': 'bf16', 'optimizer': 'adam', 'act_ckpt': False},
        '⑥ BF16 + Adam + GC':    {'dtype': 'bf16', 'optimizer': 'adam', 'act_ckpt': True},
    }
    kw = cfg[scenario]
    return compute_memory_breakdown(params_B, b, s, h, l, nh,
                                    optimizer=kw['optimizer'],
                                    dtype=kw['dtype'],
                                    act_checkpoint=kw['act_ckpt'])

# Llama-2 7B 기준
p, b, s, h, l, nh = 7, 2, 2048, 4096, 32, 32
scenarios = ['① FP32 + SGD', '② FP32 + Adam', '③ FP16 + Adam',
             '④ FP16 + Adam + GC', '⑤ BF16 + Adam', '⑥ BF16 + Adam + GC']
totals = [compute_scenario(p, b, s, h, l, nh, sc)['total'] for sc in scenarios]

fig, ax = plt.subplots(figsize=(13, 5))
sc_colors = ['#C62828', '#E53935', '#1E88E5', '#42A5F5', '#43A047', '#81C784']
bars = ax.barh(range(len(scenarios)), totals, color=sc_colors, alpha=0.85, height=0.6)

ax.axvline(x=80, color='red', ls='--', lw=2, alpha=0.8, label='H100 80GB')
ax.axvline(x=80*0.7, color='orange', ls=':', lw=1.5, label='H100 56GB (70%)')

for i, (bar, tot) in enumerate(zip(bars, totals)):
    reduction = (1 - tot / totals[0]) * 100
    label = f'{tot:.0f} GB (기준)' if i == 0 else f'{tot:.0f} GB (-{reduction:.0f}%)'
    ax.text(tot + 0.5, i, label, va='center', fontsize=9, fontweight='bold')

ax.set_yticks(range(len(scenarios)))
ax.set_yticklabels(scenarios, fontsize=10)
ax.set_xlabel('총 학습 메모리 (GB)', fontsize=11)
ax.set_title(f'Llama-2 7B 메모리 최적화 옵션 비교\n(Batch=2, Seq=2048)', fontweight='bold')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\n메모리 최적화 효과 정리:")
for sc, tot in zip(scenarios, totals):
    red = (1 - tot/totals[0]) * 100
    fits = '단일 H100 ✅' if tot <= 80*0.7 else f'H100 {int(np.ceil(tot/(80*0.7)))}대 필요'
    print(f"  {sc:<24}: {tot:>6.0f} GB (-{red:>3.0f}%) → {fits}")

## 6. 정리

### 핵심 개념 요약

| 항목 | FP32 학습 | Mixed Precision (FP16+FP32) |
|------|----------|----------------------------|
| 파라미터 | FP32 ($4P$) | FP16 ($2P$) + FP32 마스터 ($4P$) |
| 그래디언트 | FP32 ($4P$) | FP16 ($2P$) |
| Adam 상태 | FP32 ($8P$) | FP32 ($8P$) |
| **합계** | **$16P$** | **$16P$** ← 동일! |
| **연산 속도** | 기준 | 1.5~2배 빠름 (Tensor Core 활용) |

> 💡 Mixed Precision은 **메모리 절약 목적이 아니라 연산 속도 향상**이 주 목적!  
> 메모리 절약은 ZeRO, Gradient Checkpointing 등이 담당한다.

### 핵심 수식

$$M_{total} = \underbrace{(2+2+4+4+4)P}_{=16P \text{ bytes}} + \underbrace{B \cdot S \cdot H \cdot 12L \cdot \text{bytes}}_{\text{Activation}}$$

### 데이터타입 선택 가이드

| 타입 | 장점 | 단점 | 추천 용도 |
|------|------|------|----------|
| **FP32** | 최고 정밀도 | 메모리 2배, 속도 절반 | 옵티마이저 상태, 마스터 가중치 |
| **BF16** | FP32 범위 동일, 속도 2배 | FP16보다 정밀도 낮음 | **LLM 학습 권장 (A100/H100)** |
| **FP16** | 속도 2배 | Overflow 위험 (Loss Scaling 필요) | GPU가 BF16 미지원 시 |
| **FP8** | 속도 4배, 메모리 절반 | 실험적, FP32 마스터 필수 | DeepSeek-V3, H100 전용 |

### 다음 챕터 예고
**Chapter 10-02: Gradient Checkpointing** — Activation Memory를 제곱근 수준으로 줄이는 핵심 기법. Forward 결과 일부를 버리고 Backward 시 선택적으로 재계산하는 원리와 메모리-연산량 Trade-off ($O(\sqrt{L})$ vs $O(L)$)를 학습한다.